# Day 10: 畳み込みとフィルタ処理（後半）

## Learning Objectives
- エッジ検出フィルタを理解する
- 鋭敏化フィルタを実装する
- 各種フィルタを使いこなす

## エッジの数学的定義と勾配理論

エッジ検出は、画像の微分理論に基づく最も重要な画像処理技法の一つでござる。

In [ ]:
import math

# エッジ検出の数学的実装
def calculate_gradient(image, i, j):
    """勾配を計算"""
    # x方向の勾配（中央差分）
    dx = image[i][j+1] - image[i][j-1] if j > 0 and j < len(image[0])-1 else 0
    
    # y方向の勾配（中央差分）
    dy = image[i+1][j] - image[i-1][j] if i > 0 and i < len(image)-1 else 0
    
    return dx, dy

def edge_strength(image, i, j):
    """エッジの強度を計算"""
    dx, dy = calculate_gradient(image, i, j)
    # 勾配の大きさ
    return math.sqrt(dx**2 + dy**2)

def edge_direction(image, i, j):
    """エッジの方向を計算（度）"""
    dx, dy = calculate_gradient(image, i, j)
    if dx == 0:
        return 90 if dy > 0 else -90
    return math.degrees(math.atan2(dy, dx))

# テスト画像
test_image = [
    [100, 110, 120, 130, 140],
    [100, 110, 120, 130, 140],
    [100, 110, 120, 130, 140],
    [200, 210, 220, 230, 240],
    [200, 210, 220, 230, 240]
]

def print_image(image, title):
    print(f"\n{title}:")
    for row in image:
        print(' '.join(f'{x:3d}' for x in row))

print_image(test_image, "テスト画像（縦エッジ）")

# 中心位置の勾配を計算
i, j = 2, 2
dx, dy = calculate_gradient(test_image, i, j)
strength = edge_strength(test_image, i, j)
direction = edge_direction(test_image, i, j)

print(f"\n位置 ({i}, {j}) の勾配:")
print(f"dx (水平勾配): {dx}")
print(f"dy (垂直勾配): {dy}")
print(f"エッジ強度: {strength:.2f}")
print(f"エッジ方向: {direction:.2f}°")

## 高度なエッジ検出

In [ ]:
def apply_filter_with_padding(image, kernel, padding=0):
    """パディングを追加したフィルタ適用"""
    # パディング
    if padding > 0:
        padded = [[image[0][0]] * (len(image[0]) + 2*padding) for _ in range(len(image) + 2*padding)]
        for i in range(len(image)):
            for j in range(len(image[0])):
                padded[i+padding][j+padding] = image[i][j]
    else:
        padded = image
    
    rows = len(padded)
    cols = len(padded[0])
    
    # 出力画像
    filtered = [[0 for _ in range(len(image[0]))] for _ in range(len(image))]
    
    # 畳み込み
    for i in range(len(image)):
        for j in range(len(image[0])):
            value = 0
            for ki in range(-1, 2):
                for kj in range(-1, 2):
                    pixel = padded[i + padding + ki][j + padding + kj]
                    weight = kernel[ki + 1][kj + 1]
                    value += pixel * weight
            filtered[i][j] = int(value)
    
    return filtered

# Prewittフィルタ
prewitt_x = [
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
]

prewitt_y = [
    [-1, -1, -1],
    [0, 0, 0],
    [1, 1, 1]
]

prewitt_x_result = apply_filter_with_padding(test_image, prewitt_x)
prewitt_y_result = apply_filter_with_padding(test_image, prewitt_y)

print("Prewittフィルタ:")
print_image(prewitt_x_result, "Prewitt X（水平エッジ）")
print_image(prewitt_y_result, "Prewitt Y（垂直エッジ）")

# Robustエッジ検出
def robust_edge_detection(image):
    """ロバストなエッジ検出（閾値処理付き）"""
    rows = len(image)
    cols = len(image[0])
    
    # Cannyアルゴリズムの簡略版
    edges = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # 1. ガウシアンぼかし（平滑化）
    gaussian = [
        [1/16, 2/16, 1/16],
        [2/16, 4/16, 2/16],
        [1/16, 2/16, 1/16]
    ]
    blurred = apply_filter_with_padding(image, gaussian)
    
    # 2. 勾配計算
    for i in range(rows):
        for j in range(cols):
            strength = edge_strength(blurred, i, j)
            # 強度が閾値以上ならエッジと判定
            if strength > 50:  # 閾値
                edges[i][j] = min(255, int(strength))
    
    return edges

robust_edges = robust_edge_detection(test_image)
print_image(robust_edges, "ロバストなエッジ検出（閾値=50）")

## 鋭敏化フィルタ（高周波強調）

In [ ]:
# ハイパスフィルタ（高周波成分を通す）
high_pass_kernel = [
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
]

high_pass = apply_filter_with_padding(test_image, high_pass_kernel)
print_image(high_pass, "ハイパスフィルタ（高周波強調）")

# アンシャープマスク（元画像にエッジを追加）
def unsharp_mask(image, kernel, amount=1.0):
    """アンシャープマスク処理"""
    # 元画像
    original = [row[:] for row in image]
    
    # エッジ検出
    edges = apply_filter_with_padding(image, kernel)
    
    # 元画像にエッジを加算
    result = [[0 for _ in range(len(row))] for row in image]
    for i in range(len(image)):
        for j in range(len(image[0])):
            result[i][j] = min(255, original[i][j] + amount * edges[i][j])
    
    return result

# エッジ検出カーネル
edge_detect_kernel = [
    [0, -1, 0],
    [-1, 4, -1],
    [0, -1, 0]
]

unsharp = unsharp_mask(test_image, edge_detect_kernel, amount=1.0)
print_image(unsharp, "アンシャープマスク（amount=1.0）")

print("特徴:")
print("- ハイパスフィルタ: 高周波成分（エッジ、ノイズ）を強調")
print("- アンシャープマスク: 元画像にエッジを加算してシャープ化")
print("- 鋭敏化: 画像のディテールを強調する技術")

## Exercise: フィルタの応用

In [ ]:
# メディアンフィルタ（ノイズ除去）
def median_filter(image, size=3):
    """メディアンフィルタ（ノイズ除去）"""
    rows = len(image)
    cols = len(image[0])
    
    # パディング
    padding = size // 2
    padded = [[0 for _ in range(cols + 2*padding)] for _ in range(rows + 2*padding)]
    for i in range(rows):
        for j in range(cols):
            padded[i+padding][j+padding] = image[i][j]
    
    # メディアンフィルタ適用
    filtered = [[0 for _ in range(cols)] for _ in range(rows)]
    
    for i in range(rows):
        for j in range(cols):
            # 近傍のピクセルを取得
            neighborhood = []
            for ki in range(-padding, padding+1):
                for kj in range(-padding, padding+1):
                    neighborhood.append(padded[i+padding+ki][j+padding+kj])
            # 中央値を取得
            neighborhood.sort()
            filtered[i][j] = neighborhood[len(neighborhood)//2]
    
    return filtered

# ノイズ付き画像を作成
noisy_image = [row[:] for row in test_image]
import random
for i in range(len(noisy_image)):
    for j in range(len(noisy_image[0])):
        if random.random() < 0.1:  # 10%の確率でノイズ
            noisy_image[i][j] = random.choice([0, 255])

print("ノイズ付き画像:")
print_image(noisy_image, "ノイズ付き")

denoised = median_filter(noisy_image)
print_image(denoised, "メディアンフィルタで除去")

print("\nメディアンフィルタの特徴:")
print("- 外れ値の影響を受けない（ロバスト）")
print("- salt-and-pepperノイズに効果的")
print("- エッジを保持しながらノイズ除去")

**お疲れ様でした！** Day 10完了！

## Self-Check
- [ ] 勾配理論を理解した
- [ ] 各種エッジ検出フィルタを実装した
- [ ] 鋭敏化フィルタを実装した
- [ ] メディアンフィルタでノイズ除去を試した

**次回（Day 11）からは色彩理論とカラー画像処理を学びます。**